<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Doda-now-kolumn---ilo-nagrd-dla-danego-aktora/colabs/Wczytanie_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git
!mv /content/BigData2022-actors/* .
!mv /content/BigData2022-actors/.* .
!rmdir /content/BigData2022-actors/

In [ ]:
#!git checkout

In [ ]:
!chmod +x setup_sparka.sh
!./setup_sparka.sh

In [ ]:
import pandas as pd
from lib.pyspark_init import create_spark_context, load_data
from lib.const import JOINED_DATA

In [ ]:
spark = create_spark_context()
spark

In [ ]:
!chmod +x download_data.sh
!./download_data.sh

In [ ]:
%%time
data = load_data(spark)
data.show(3)

In [ ]:
%%time
data.write.parquet(JOINED_DATA)